In [1]:
import boto3

In [2]:
AWS_KEY_ID = 'AWS KEY'
AWS_SECRET = 'SECRET'
s3 = boto3.resource('s3', aws_access_key_id = AWS_KEY_ID,
                    aws_secret_access_key = AWS_SECRET)

In [3]:
try:
    s3.create_bucket(Bucket='kesterzhou14848hw3', CreateBucketConfiguration={'LocationConstraint': 'us-west-2'})
except Exception as e:
    print(e)
    
bucket = s3.Bucket("kesterzhou14848hw3")
bucket.Acl().put(ACL='public-read')

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


{'ResponseMetadata': {'RequestId': 'R8ZXNQ4Q2WKDRDA0',
  'HostId': 'dxZpKCyzVtZL+VjEH7ebwPLPvgZh1jiYGH+6YtlycnCrX1MB8x0vI5Mr6Eri8jCGb7d4fwCWNfQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'dxZpKCyzVtZL+VjEH7ebwPLPvgZh1jiYGH+6YtlycnCrX1MB8x0vI5Mr6Eri8jCGb7d4fwCWNfQ=',
   'x-amz-request-id': 'R8ZXNQ4Q2WKDRDA0',
   'date': 'Fri, 15 Oct 2021 19:45:19 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [4]:
s3.Object('kesterzhou14848hw3', 'boyo.jpg').put(Body=open('boyo.jpg', 'rb'))
s3.Object('kesterzhou14848hw3','boyo.jpg').Acl().put(ACL='public-read')


{'ResponseMetadata': {'RequestId': 'QXWMPPJ8JXYJ7BNQ',
  'HostId': 'MVbT7u9xuyU0GixjQEtpbu4A0SH5rh58tJAxvWjglEciw0AD1VKT331aEGktnWkK7i9beOP5Z5E=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'MVbT7u9xuyU0GixjQEtpbu4A0SH5rh58tJAxvWjglEciw0AD1VKT331aEGktnWkK7i9beOP5Z5E=',
   'x-amz-request-id': 'QXWMPPJ8JXYJ7BNQ',
   'date': 'Fri, 15 Oct 2021 19:45:21 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [13]:
# Now we create the DynamoDB table, first we create the resource
dyndb = boto3.resource('dynamodb',
region_name='us-west-2', aws_access_key_id=AWS_KEY_ID, aws_secret_access_key=AWS_SECRET )

In [14]:
try:

    table = dyndb.create_table(
    TableName='DataTable', KeySchema=[ { 'AttributeName': 'PartitionKey', 'KeyType': 'HASH' }, { 'AttributeName': 'RowKey', 'KeyType': 'RANGE' }
        ], 
        AttributeDefinitions=[
            { 
                'AttributeName': 'PartitionKey', 
                'AttributeType': 'S' 
            }, 
            {
                'AttributeName': 'RowKey',
        'AttributeType': 'S' },
        ], 
        ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5 
        }
    ) 

except Exception as e:
    print (e)
    
    #if there is an exception, the table may already exist. 
    table = dyndb.Table("DataTable")

In [15]:
#wait for the table to be created 
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [16]:
print(table.item_count)

0


In [17]:
import csv

In [31]:
with open('experiments.csv', 'rt', encoding = "utf-8") as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    next(csvf)
    for item in csvf:
        print(item)
        body = open(item[4], 'rb')
        s3.Object('kesterzhou14848hw3', item[4]).put(Body=body )
        md = s3.Object('kesterzhou14848hw3', item[4]).Acl().put(ACL='public-read')
        url = " https://s3-us-west-2.amazonaws.com/kesterzhou14848hw3/"+item[4]
        metadata_item = {'PartitionKey': item[4], 'RowKey': item[0],
        'description' : item[4], 'date' : item[2], 'url':url}

        try:
            table.put_item(Item=metadata_item) 
        except:
            print("item may already be there or another failure")

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [32]:
response = table.get_item( Key={ 'PartitionKey': 'exp1.csv', 'RowKey': '1' } ) 
item = response['Item'] 
print(item)

{'PartitionKey': 'exp1.csv', 'RowKey': '1', 'date': '52', 'description': 'exp1.csv', 'url': ' https://s3-us-west-2.amazonaws.com/kesterzhou14848hw3/exp1.csv'}


In [33]:
response

{'Item': {'PartitionKey': 'exp1.csv',
  'RowKey': '1',
  'date': '52',
  'description': 'exp1.csv',
  'url': ' https://s3-us-west-2.amazonaws.com/kesterzhou14848hw3/exp1.csv'},
 'ResponseMetadata': {'RequestId': 'PI1RBHQRLQMBL6IA1J3458713RVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 15 Oct 2021 20:32:21 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '188',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'PI1RBHQRLQMBL6IA1J3458713RVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '1259132623'},
  'RetryAttempts': 0}}